# data pre-processing: combine related fields based on their definition and drop the original

In [69]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [70]:
train['source'] = 'train'
test['source'] = 'test'
combine = pd.concat([train,test]).reset_index().drop('index', axis = 1)


In [71]:
# reformat Exterior1st and Exterior2nd
ext1 = pd.get_dummies(combine.Exterior1st) # df with dummify Exterior1st
ext2 = pd.get_dummies(combine.Exterior2nd).rename(columns={'Brk Cmn':'BrkComm', 'CmentBd':'CemntBd', 'Wd Shng':'WdShing'}) # df with dummify Exterior2nd, correct typos in ext2
ext = ext1.add(ext2, fill_value=0).astype('uint8').replace(2,1).add_prefix('ext_')  # combine dummified df and add a prefix 
combine = pd.concat([combine, ext], axis=1).drop(['Exterior1st', 'Exterior2nd'], axis=1)


In [72]:
# combine BsmtFullBath, BsmtHalfBath (number of type of bathroom in the basement)
combine['bathtot'] = train.BsmtFullBath + train.BsmtHalfBath * 0.5
combine = combine.drop(['BsmtFullBath', 'BsmtHalfBath'], axis=1)

In [73]:
# reformat PoolQC, PoolArea
poolinfo = pd.concat([combine.PoolArea,  pd.get_dummies(combine.PoolQC)], axis =1) # df with dummified PoolQC columns + pool area
pool = poolinfo.apply(lambda row: row.replace(1, row['PoolArea']), axis=1) # cast pool area into dummified columns
combine = pd.concat([combine, pool], axis =1).drop(['PoolArea', 'PoolQC'], axis = 1)


In [74]:
# reformat Bsmt 
# BsmtType1
bsmt1info = pd.concat([combine.BsmtFinSF1, pd.get_dummies(combine.BsmtFinType1)], axis = 1) # df with dummified BsmtFinType1 columns + BsmtFinSF1
bsmt1 = bsmt1info.apply(lambda row: row.replace(1, row['BsmtFinSF1']), axis = 1) # cast BsmtFinSF1 into dummified columns (unfinished SF will be handled in the BsmtFinType2 section since we are merging type1 and type2 at the end)

# BsmtType2
combine.BsmtFinSF2 = combine[['BsmtFinSF2','BsmtUnfSF']].apply(lambda row: row.replace(0, row['BsmtUnfSF']), axis = 1) # merge unfinished SF into BsmtFinSF2 column so all the unfinished SF + BsmtFinSF2 are in one column and can be cast onto the dummy columns
bsmt2info = pd.concat([combine.BsmtFinSF2, pd.get_dummies(combine.BsmtFinType2)], axis = 1) # df with dummified BsmtFinType2 columns + BsmtFinSF2 (BsmtFinSF2 column here included both SF2 and unfinished SF)
bsmt2 = bsmt2info.apply(lambda row: row.replace(1, row['BsmtFinSF2']), axis = 1) # cast BsmtFinSF2 into dummified columns

bsmt = bsmt1.add(bsmt2, fill_value=0).drop(['BsmtFinSF1','BsmtFinSF2'], axis = 1).add_prefix('bsmt_') # combine dummified BsmtType1 and BsmtType2, drop original columns, add prefix 
combine = pd.concat([combine, bsmt], axis = 1).drop(['BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'], axis = 1)



In [75]:
# reformat masonry 
#MasVnrType, MasVnrArea
masonryinfo = pd.concat([combine.MasVnrArea, pd.get_dummies(combine.MasVnrType)], axis = 1) # df with dummified masonry columns + masonryArea
masonry = masonryinfo.apply(lambda row: row.replace(1, row['MasVnrArea']), axis = 1) # cast masonryArea into dummified columns

combine = pd.concat([combine, masonry], axis = 1).drop('MasVnrArea', axis = 1)


In [76]:
# combine different porch areas (OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch) and create a new col with total porch area
combine['porchtot'] = sum([combine.OpenPorchSF, combine.EnclosedPorch, combine['3SsnPorch'], combine.ScreenPorch])


In [77]:
combine.to_csv('combine.csv')


In [ ]:
# save file to picked obj
df_train.to_pickle("df_train.pkl")
df_test.to_pickle("df_test.pkl")
df.to_pickle("df.pkl")


# create na for dummy 
# factorized categorical col